In [1]:
!pip -q install lightgbm optuna pandas openpyxl --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 131.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 22.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.0 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.


In [4]:
# ===== 1. 匯入 =====
import pandas as pd, numpy as np, warnings
warnings.filterwarnings('ignore')

# ===== 2. 路徑 =====
FX_PATH = '/content/processed_data.csv'      # 已上傳到 Colab 的位置
MM_PATH = '/content/merged_data.xlsx'

# ===== 3. 清洗函式 =====
def preprocess(fx_path, mm_path):
    fx = (pd.read_csv(fx_path, parse_dates=['Date'])
            .set_index('Date'))

    mm = (pd.read_excel(mm_path, parse_dates=['Time'])
            .rename(columns={'Time':'Date'})
            .set_index('Date')
            .apply(pd.to_numeric, errors='coerce'))

    # 月頻 ➜ 日頻
    mm = mm.resample('D').ffill()

    df = fx.join(mm, how='left')

    # 時間特徵
    df['year']  = df.index.year
    df['month'] = df.index.month
    df['dow']   = df.index.dayofweek

    # Lag
    df['lag_1']  = df['TWD_USD'].shift(1)
    df['lag_5']  = df['TWD_USD'].shift(5)
    df['lag_20'] = df['TWD_USD'].shift(20)

    # 三段式補值
    df = (df.interpolate(method='time', limit_direction='both')
            .fillna(method='ffill')
            .fillna(method='bfill'))

    return df.dropna()

# ===== 4. 產生 data  =====
data = preprocess(FX_PATH, MM_PATH)
print('data ready:', data.shape)

# ===== 5. 切訓練／測試  =====
train_df = data.loc[:'2023-12-31']
test_df  = data.loc['2024-01-01':]

print(train_df.index.min(), '→', train_df.index.max(),
      '| train len:', len(train_df))
print(test_df.index.min(),  '→', test_df.index.max(),
      '| test  len:', len(test_df))


data ready: (6630, 24)
2000-01-03 00:00:00 → 2023-12-29 00:00:00 | train len: 6260
2024-01-01 00:00:00 → 2025-05-30 00:00:00 | test  len: 370


In [7]:
# ===== 6. 變數定義 =====
import lightgbm as lgb, optuna, numpy as np, time, warnings
warnings.filterwarnings('ignore')

TARGET   = 'TWD_USD'                             # 目標欄位
FEATURES = [c for c in data.columns if c != TARGET]   # 其餘全部拿來當 X

print('X dims =', len(FEATURES), '→', FEATURES[:6], '...')

# ===== 7. LightGBM Dataset =====
train_set = lgb.Dataset(train_df[FEATURES], label=train_df[TARGET])
valid_set = lgb.Dataset(test_df[FEATURES],  label=test_df[TARGET],
                        reference=train_set)

# ===== 8. Optuna：20 trials  (≈ 1 分鐘) =====
def objective(trial):

    params = {
        'objective':'regression',
        'metric':'rmse',
        'learning_rate': trial.suggest_float('lr', 0.01, 0.12, log=True),
        'num_leaves':      trial.suggest_int('nl', 31, 128),
        'feature_fraction':trial.suggest_float('ff', 0.6, 1.0),
        'bagging_fraction':trial.suggest_float('bf', 0.6, 1.0),
        'bagging_freq':    trial.suggest_int('bq', 1, 10),
        'seed':42, 'verbosity':-1
    }

    gbm = lgb.train(
        params, train_set,
        num_boost_round=1200,
        valid_sets=[valid_set],
        callbacks=[lgb.early_stopping(50, verbose=False)]
    )

    preds = gbm.predict(test_df[FEATURES], num_iteration=gbm.best_iteration)
    rmse  = np.sqrt(((preds - test_df[TARGET])**2).mean())
    return rmse


print('⏱  Optuna hyper-search (20 trials)…')
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20, show_progress_bar=False)

best_params = study.best_params | {
    'objective':'regression',
    'metric':'rmse',
    'verbosity':-1,
    'seed':42
}
print('Best RMSE trial =', study.best_value)

# ===== 9. 用最佳參數重新訓練  (≈ 30–40 秒) =====
model = lgb.train(
    best_params, train_set,
    num_boost_round=1800,
    valid_sets=[valid_set],
    callbacks=[lgb.early_stopping(100, verbose=True)]
)

# ===== 10. 2024 Out-Sample 評估 =====
pred_2024 = model.predict(test_df[FEATURES], num_iteration=model.best_iteration)
rmse_2024 = np.sqrt(((pred_2024 - test_df[TARGET])**2).mean())
print(f'🎯 2024 RMSE = {rmse_2024:.4f}')

# ===== 11. 儲存模型 =====
MODEL_PATH = '/content/usd_twd_lgb_2024.txt'
model.save_model(MODEL_PATH)
print('✅  model saved →', MODEL_PATH)


[I 2025-06-24 13:21:21,567] A new study created in memory with name: no-name-7e7bc70c-d68b-462e-bb69-aac6b907d4a4


X dims = 23 → ['CNY_USD', 'JPY_USD', 'USD_EUR', 'KRW_USD', 'TWD_CNY', 'TWD_JPY'] ...
⏱  Optuna hyper-search (20 trials)…


[I 2025-06-24 13:21:22,261] Trial 0 finished with value: 0.142319851784145 and parameters: {'lr': 0.01741342381078447, 'nl': 68, 'ff': 0.8081818019379009, 'bf': 0.6712505404907234, 'bq': 8}. Best is trial 0 with value: 0.142319851784145.
[I 2025-06-24 13:21:22,614] Trial 1 finished with value: 0.1682397422139672 and parameters: {'lr': 0.06803999968553286, 'nl': 94, 'ff': 0.6154022357391763, 'bf': 0.724653213536561, 'bq': 4}. Best is trial 0 with value: 0.142319851784145.
[I 2025-06-24 13:21:23,339] Trial 2 finished with value: 0.13094410360090905 and parameters: {'lr': 0.029712770586701853, 'nl': 102, 'ff': 0.9635755068041761, 'bf': 0.8387800861126722, 'bq': 7}. Best is trial 2 with value: 0.13094410360090905.
[I 2025-06-24 13:21:23,546] Trial 3 finished with value: 0.1748223980261916 and parameters: {'lr': 0.09891522593830428, 'nl': 70, 'ff': 0.6523473275488557, 'bf': 0.8743154572949672, 'bq': 4}. Best is trial 2 with value: 0.13094410360090905.
[I 2025-06-24 13:21:23,699] Trial 4 fin

Best RMSE trial = 0.1285712903830301
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[36]	valid_0's rmse: 0.128149
🎯 2024 RMSE = 0.1281
✅  model saved → /content/usd_twd_lgb_2024.txt


In [9]:
# =============================================
# 0️⃣ 準備：必要套件
# =============================================
!pip -q install lightgbm pandas openpyxl --upgrade

import pandas as pd, numpy as np, lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# =============================================
# 1️⃣ 路徑 & 參數
# =============================================
FX_PATH    = '/content/processed_data.csv'
MM_PATH    = '/content/merged_data.xlsx'
MODEL_PATH = '/content/usd_twd_lgb_2024.txt'      # 先前儲存的模型檔
TARGET     = 'TWD_USD'

# =============================================
# 2️⃣ 資料同前：快速預處理（簡化版）
# =============================================
def preprocess(fx_path, mm_path):
    fx = (pd.read_csv(fx_path, parse_dates=['Date'])
            .set_index('Date'))

    mm = (pd.read_excel(mm_path, parse_dates=['Time'])
            .rename(columns={'Time':'Date'})
            .set_index('Date')
            .apply(pd.to_numeric, errors='coerce')
            .resample('D').ffill())

    df = fx.join(mm, how='left')
    df['year']  = df.index.year
    df['month'] = df.index.month
    df['dow']   = df.index.dayofweek
    df['lag_1']  = df[TARGET].shift(1)
    df['lag_5']  = df[TARGET].shift(5)
    df['lag_20'] = df[TARGET].shift(20)

    df = (df.interpolate(method='time', limit_direction='both')
            .fillna(method='ffill')
            .fillna(method='bfill')
            .dropna())
    return df

data = preprocess(FX_PATH, MM_PATH)
train_df = data.loc[:'2023-12-31']
test_df  = data.loc['2024-01-01':]

FEATURES = [c for c in data.columns if c != TARGET]

# =============================================
# 3️⃣ 載入模型並預測 2024
# =============================================
model = lgb.Booster(model_file=MODEL_PATH)

pred_2024 = model.predict(test_df[FEATURES], num_iteration=model.best_iteration)
test_df = test_df.copy()          # 避免 SettingWithCopyWarning
test_df['pred'] = pred_2024

# =============================================
# 4️⃣ 指標計算
# =============================================
rmse = np.sqrt(mean_squared_error(test_df[TARGET], test_df['pred']))
mae  = mean_absolute_error(test_df[TARGET], test_df['pred'])
mape = (np.abs((test_df[TARGET] - test_df['pred']) / test_df[TARGET]).mean()) * 100
r2   = r2_score(test_df[TARGET], test_df['pred'])

direction_acc = (np.sign(test_df['pred'].diff()) == np.sign(test_df[TARGET].diff())).mean()

print(f"🧮 2024 Metrics")
print(f"    RMSE  = {rmse:.5f}")
print(f"    MAE   = {mae:.5f}")
print(f"    MAPE  = {mape:.3f}%")
print(f"    R²    = {r2:.4f}")
print(f"    DirAcc= {direction_acc:.3%}")

# =============================================
# 5️⃣ 結果匯出（選擇性）
# =============================================
OUTPUT_CSV = '/content/usd_twd_pred_2024.csv'
test_df[[TARGET, 'pred']].to_csv(OUTPUT_CSV)
print("📄  詳細預測已儲存：", OUTPUT_CSV)

# 下載方法（Colab）：點左側 Files 面板 → 右鍵 Download


🧮 2024 Metrics
    RMSE  = 0.12815
    MAE   = 0.07913
    MAPE  = 0.248%
    R²    = 0.9682
    DirAcc= 37.027%
📄  詳細預測已儲存： /content/usd_twd_pred_2024.csv
